# TUTORIEL  SCRAPING FACEBOOK 
###  Cas: décès du président Jacques Chirac en septembre 2019

| Prénoms       |     Nom         |     Email          |                                     |     Lieu    |   Date   |
| ------------- |: -------------: |--------------------|                                     | ------------|----------|
| Amadou lamarana      | DIALLO               | mailsdiallo@gmail.com|                       | Paris       | 16-06-2021|


<hr>
 

### LES ÉTAPES :

#### A ) Importer les modules nécessaires

#### B ) Configuration  et lancement du navigateur webdriver

#### C ) La connexion sur le compte facebook pour récupérer les données ou posts

#### D) Parcours de toute la page avec un système de scrolling afin de récuperer tous les posts pertinents par rapport à notre thématique "le décès du président Jacques Chirac" et les résultats sont affichés sur la page

#### E) Récupération des articles des tous les posts sur la page de résultat 

#### F) Récupérer les liens des posts

#### G) Création d'un DataFrame df pour la persistance des liens

#### H) Sauvegarder les liens dans un fichier au format csv

#### I) Utilisation du package facebook-scraper

#### J) MongoDB pour la sauvegarde des données en local

#### A ) Importer les modules nécessaires

    * Selenuim :https://selenium-python.readthedocs.io/index.html

    * time et math uuid sont natifs à python lors de son l'installation

    * pandas: https://pandas.pydata.org/

    * cleantext : https://pypi.org/project/clean-text/


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import math
import uuid
##import pandas 
import pandas as pd
from cleantext import clean

#### B ) Configuration  et lancement du navigateur webdriver


##### Identifiant de connexion et mot de passe .. à personnaliser

In [2]:
identifiant ="your_username"
mot_de_passe ="your_password"
credentials= (identifiant, mot_de_passe)

In [4]:
""" Nous avons la configuration du webdriver en local avec 
les options élémentaires.
"""
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_argument('--incognito')
chrome_options.add_experimental_option("prefs",prefs)
"""
lien du webdriver :chrome
"""
exec_path = "../chromedriver"
"""
Exécution du webdriver
"""
browser = webdriver.Chrome(executable_path=exec_path, options=chrome_options)
"""
Accéder au site facebook
"""
browser.get("https://www.facebook.com")

"""
Acceptation des cookies pour continuer la simulation de connexion
"""
WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Tout accepter']"))).click()

#### C ) La connexion sur le compte facebook pour récupérer les données ou posts
La fonction login() aide à se connecter en utilisant des paramètres de connexion valides. 

In [5]:
def login():
    """
    Nous definissons d'une fonction login afin de se connecter sur facebook 
    avec les credentials ci dessous.
    Les taches nécessaires sont majoirtairement automatiques. 
    Le système reconnait les champs requis et fait lui même le remplissage. 
    """
    #target username
    """
    Detecter puis Renseigner l'identifiant (email et mot de passe)
    """
    email = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
    password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))
    """
    Toujours vider les champs inputs grace au méthode clear() avant le remplissage
    """
    email.clear()
    #email = driver.find_element_by_id("email")
    identifiant =credentials[0]
    mot_de_passe =credentials[1]
    """
    Renseigner le username
    """
    email.send_keys(identifiant)
    #Password = driver.find_element_by_id("pass")
    password.clear()
    """
    Renseigner le mot de passe
    """
    password.send_keys(mot_de_passe)
    button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    """
    connexion effectuée.. puis libérer 
    """
    pass 


#### La connexion proprement dite

In [6]:
"""
Puis action de se connecter sur Facebook 
"""
login()


#### D) Parcours de toute la page avec un système de scrolling afin de récuperer tous les posts pertinents par rapport à notre thématique "le décès du président Jacques Chirac" et les résultats sont affichés sur la page

In [11]:
"""
Les données sont filtrées d'avance sur Facebook et classé par Année, Publication publiques, etc. 
Nous allons définir le lien essentiel pour le résultat directement, 
"""
url_complet='https://www.facebook.com/search/posts?q=le%20d%C3%A9c%C3%A9s%20du%20pr%C3%A9sident%20%20jacques%20chirac&filters=eyJycF9hdXRob3I6MCI6IntcIm5hbWVcIjpcIm1lcmdlZF9wdWJsaWNfcG9zdHNcIixcImFyZ3NcIjpcIlwifSIsInJwX2NyZWF0aW9uX3RpbWU6MCI6IntcIm5hbWVcIjpcImNyZWF0aW9uX3RpbWVcIixcImFyZ3NcIjpcIntcXFwic3RhcnRfeWVhclxcXCI6XFxcIjIwMTlcXFwiLFxcXCJzdGFydF9tb250aFxcXCI6XFxcIjIwMTktMVxcXCIsXFxcImVuZF95ZWFyXFxcIjpcXFwiMjAxOVxcXCIsXFxcImVuZF9tb250aFxcXCI6XFxcIjIwMTktMTJcXFwiLFxcXCJzdGFydF9kYXlcXFwiOlxcXCIyMDE5LTEtMVxcXCIsXFxcImVuZF9kYXlcXFwiOlxcXCIyMDE5LTEyLTMxXFxcIn1cIn0ifQ%3D%3D'

"""
Nous tentons d'accéder au lien "url_complet"
"""
browser.get(url_complet)

"""methode scrolling pour récuperer plus de données dans la page"""

last_height = browser.execute_script("return document.body.scrollHeight") 
"""
Parcours autant que nécessaire jusqu'a atteindre le fond de la page
"""
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    """
    Attendre 2 secondes '2 s' avant chaque scroll vers le bas
    """
    time.sleep(2)
    """
    Recuperer la taille courante du frame
    """
    new_height = browser.execute_script("return document.body.scrollHeight")
    """
    Si la taille courante est égale à la taille maximale alors sortir 
    de boucle grâce une instrcution break
    """
    if new_height == last_height:
        break
    last_height = new_height

#### E) Récupération des articles des tous les posts sur la page de résultat 

In [12]:
"""
recuepérer les webElement(qui contient les liens informations du post) des posts de la page affichée
"""
#recherche =browser.find_elements_by_xpath('//*[@id="mount_0_0_wk"]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]')
recherche = browser.find_elements(By.TAG_NAME, 'div[role="article"]')

In [40]:
recherche

#### F) Récupérer les liens des posts

In [14]:
"""
Nous allons récupérer les liens des posts pertinents chargés dans une liste
"""
data = [] ## liste des liens
"""
Boucle sur les posts pertinents déja existants
"""
for r in recherche:
    """
    les balises "a" contenant les liens des posts. ce que nous cherchons: 'anchors'
    """
    
    anchors = r.find_elements_by_tag_name('a[role="link"]')[-1]
    "recuperer la valeur de href: lien du post"
    link = anchors.get_attribute('href')
    
    """
    Ne récuperer que les liens posts
    """
    if "posts" in link:
        data.append(link)

#### G) Création d'un DataFrame df pour la persistance des liens

In [15]:
"""
Stocker les données dans un dataframe afin d'une prochaine utilisation
"""
df = pd.DataFrame(data, columns=['Link'])

In [16]:
"""
Suppresion des duplicats
"""
df.drop_duplicates(inplace=True)

In [41]:
df

#### H) Sauvegarder les liens dans un fichier au format csv

In [20]:
df.to_csv("Liens_Post_Facebook.csv")

#### I) Utilisation du package facebook-scraper

Lien web: https://pypi.org/project/facebook-scraper/


Installation :  pip install facebook-scraper

In [23]:
from facebook_scraper import get_posts

##### 1) Renseigner le login et le mot du compte qui sera utilisé pour se connecter sur Facebook

##### 2) Module de gestion de fichier et du telechargement des images en local

In [24]:
import os
import wget

#### J) Recupération des données sur Facebook  à partir des données
###### Les données à utiliser:  
    *** notre liste d'url disponible sur avec la variable data OU  
    *** récuperer les liens au niveau du dataframe  OU  
    *** par le fichier csv

In [39]:
data
#df= pd.read_csv('Liens_Post_Facebook.csv')
#data = list(df['Link'])


##### Utilisation du package facebook-scraper

In [28]:
"""
dictionnaire posts pour enregistrer les données..
"""
posts ={'texts':[],'image_link':[],'image_local_path':[], 'comments':[]}
"""
boucle sur les liens de données recupérées
"""
for post in get_posts(post_urls=data, credentials=credentials,options={"comments": True}):
    """
        text_NA: donnée à enregistrer en cas d'absence de texte
    """
    text_NA ='XXX'
    """
        si le texte du post est défini
    """
    if post['text'] is not None:
        """
            nettoyage du texte et ajout sur la liste du dictionnaire posts['texts']
        """
        posts['texts'].append(clean(post['text'].strip(),normalize_whitespace=True, no_line_breaks=True, lang='fr'))
    else:
        """
            si pas de text
        """
        posts['texts'].append(text_NA)
    """
        Si l'image est définie alors traiement et gestion de l'image
    """
    if post['image'] is not None:
        
        posts['image_link'].append(post['image'])
        """
            Télécharger l'image si elle est disponible
            # 1 : dossier de travail
            # 2 : mention du dossier de sauvegarde des images en local (dossier images_facebook)
            # 3 : si le dossier n'existe pas, alors le créer
            # 4 : filename du fichier avec l'id du post_id du post sur facebook combiné à un uuid() pour eviter les duplications
        """
        path = os.getcwd() # 1
        path = os.path.join(path, 'images_facebook') # 2
        if  os.path.exists(path)== False: # 3
            os.mkdir(path) # 3
        rename_file = os.path.join(path, str(post['post_id'])+str(uuid.uuid1())+'.png') # 4
        """
            Si le fichier n'existe pas en répertoire 
        """
        if os.path.exists(rename_file) == False:
            try:
                """
                    tentative de telechargement de l'image et sauvgarde sur un répertoire crée à l'occasion
                """
                wget.download(post['image'], rename_file)
                
                posts['image_local_path'].append(rename_file)
                
            except:
                """Cas où le téléchargement a échoué, j'ai jugé nécessaire de mettre en placeholder, 
                une image 'no_image' pour sympbiliser l'absence de l'élément
                """
                posts['image_local_path'].append(path+'/images_facebook/no_image.png')
                
    else:
        """
             Si il n'y a pas d'image.alors renseigner XXX sur le lien de notre dictioonaire. et pour lien d'image en local, l'image par défaut
        """
        posts['image_link'].append('XXX')
        posts['image_local_path'].append(path+'/images_facebook/no_image.png')
        
    """
        Si les commentaires existent alors: 
            1) Opération de nettoyage du texte
            2) Mettre tous les commentaires du post dans une même liste pour faciliter le traitement futur
    """
    if post['comments_full'] is not None:
        comment_sanitized =[]
        """
            Parcourir les commentaires du post
        """
        for comment in post['comments_full']:
            """
            Nettoyage du texte des commentaires et sauvegader dans une liste "comment_sanitized"
            """
            comment_sanitized.append(clean(comment['comment_text'],normalize_whitespace=True, no_line_breaks=True, lang='fr'))
        
        """
            Une fois la boucle terminée, la liste des commentaires nettoyés est ajouté dans notre dictionnaire 'posts' au niveau de la liste "comments"
        """
        posts['comments'].append(comment_sanitized)
        """
            Vider la liste des commentaires nettoyés du post et passage pour le prochain post
        """
        comment_sanitized =[]
    else:
        """
            Si il n'y a pas de commentaire pour le post alors .. la valeur 'XXX' lui est attribuée
        """
        posts['comments'].append('XXX')

##### Réorganisation des commentaires sous format liste de dictionnaires
<code>    post :{
                'text': le_texte,
                'image': l_image_local,
                'comments': [{comment1},{comment_2},......]
            } </code>

In [31]:
"""
Nous allons organiser les commentaires pour chaque post, sous le format ci-dessus. 
pour facilter la representaion et le stockage en mongoDB
"""
comments = {}
all_comments =[]
for k in range(len(posts['comments'])):
    for com in posts['comments'][k]:
        comment = {'comment_text': com }
        all_comments.append(comment)
    comments[k] = all_comments
    all_comments = []


#### J) MongoDB pour la sauvegarde des données en local

In [33]:
"""
Nous allons utiliser le package pymongo pour interagir avec MongoDB installé en local.
Les paramètres sont ceux définis par défaut .. sans user sans mot de passe
pprint sert à un afficahge pour une facilité de lecture et de visualisation.
"""
from pymongo import MongoClient
from pprint import pprint
"""
Etape1: Se connecter à la base MongoDB
"""
client = MongoClient(port=27017)
"""
Création de la base de données 'db_name'
"""
db=client.db_name

"""
Etape 2: Schema de données
"""
n = len(posts['texts']) ## nombre de posts 
"""
parcours de 0 à n-1
le but de cet ensemble d'istruction est de définir un dictionnaire simple pour chaque post pour des rasions
 **** facilité l'affichage et les insertions dans la base de données mongodb
"""
for i in range(n):
    post ={
        'text' : posts["texts"][i],
        'image':posts['image_local_path'][i],
        'comments':comments[i]
    }
    #print(comments[i])
    """
    Etape 3: Insertion de chaque post directement dans la base de données db.db_name 
    db_name est le nom de la base de données
    """
    result=db.db_name.insert_one(post)


#### Liste de bases de données en local sur mongoDB

In [36]:
client.database_names()

#### Petite manipulation pour la visualisation des données

In [35]:
"""
Affichage des documents: db_name est le nom de la base de données
"""
for d in db.db_name.find():
    pprint(d)

#### Nombre de documents dans la base

In [37]:
mesdonnes = db.db_name

In [38]:
mesdonnes.count_documents({})


60


<div class='text text-primary'> Nos documents sont bien enregistrés dans la base mongoDB </div>

### CONCLUSION

Nous avons implémenté des scripts à l'aide Selenuim, et le package facebook-scraper pour la recherche de l'information à savoir les posts, les images et les commentaires associés sur Facebook sur la thématique  'le décès du président Jacques Chirac' en septembre 2019.
Nous avons ensuite pû implémenter le script pour l'enregistrement dans une base de données MongoDB. 
Nous avons pas utilisé une application Facebook pour se servir des GraphAPI de Facebook, vu que la thématique demandé n'est pas lié à une page quelconque. Ce qui nous a amené à implementer un script de scrapping à l'aide Sélénuim.